In [52]:
import pandas as pd 
import requests
import json 
from datetime import datetime, timedelta

df = pd.read_csv('transactions.csv')

df.head()

,product_code,customer_code,market_code,order_date,sales_qty,sales_amount,currency
0,Prod001,Cus001,Mark001,2017-10-10,100,41241,INR
1,Prod001,Cus002,Mark002,2018-05-08,3,-1,INR
2,Prod002,Cus003,Mark003,2018-04-06,1,875,INR
3,Prod002,Cus003,Mark003,2018-04-11,1,583,INR
4,Prod002,Cus004,Mark003,2018-06-18,6,7176,INR


You can query the ECB exchange rate API like this:

In [43]:
r = requests.get('https://api.exchangeratesapi.io/history?start_at=2018-01-01&end_at=2018-01-02&symbols=INR&base=USD')

js = json.loads(r.text)
js

{'rates': {'2018-01-02': {'INR': 63.4898466639}},
 'start_at': '2018-01-01',
 'base': 'USD',
 'end_at': '2018-01-02'}

In [26]:
#extract the relevant values
js['rates']['2018-01-02']['INR']

63.4898466639

In [67]:
def converttoINR(row): 
    if row['currency'] == 'USD': 
        startdate = row['order_date']
        enddate = datetime.strptime(startdate, '%Y-%m-%d') + timedelta(days=1)
        enddate = enddate.strftime('%Y-%m-%d')

        r = requests.get('https://api.exchangeratesapi.io/history?start_at={start}&end_at={end}&symbols=INR&base=USD'.format(start=startdate, end=enddate))
        rate = json.loads(r.text)['rates'][startdate]['INR']
        return row['sales_amount'] * rate
    else: 
        return row['sales_amount']
    

In [68]:
df['INR sales'] = df.apply(converttoINR, axis=1)

In [73]:
df = df.drop(columns=['sales_amount', 'currency'])

In [75]:
df.to_csv('transactions_cleaned.csv', index=False)